## Experimental Work and The Transferability Principle
```In this exercise you will experience with real experimental work and will meet a very interesting issue - adversarial AI and the transferability principle. You will define experiments and measures for success, and will execute those experiments. It is of great importance that you will discuss this exercise with your tutor, even during the work on the exercise.```

```~Ohad Amosi & Ittai Haran```

```Read the paper "Adversarial Examples Are Not Bugs, They Are Features", which you can find in this exercise directory. Read it thoroughly. Make sure you understand how the datasets of experiments #1 and #2 were generated. As you might tell, the paper reports very interesting results.```

```If you are not familiar with the concept of model distillation, read about it. There arise the question, how can we be sure that the "non-robust" features described in the paper are real? Could it be that the effect the paper measures is a model distillation? Think about it: In both the experiments the authors used a trained robust network to label images, on which another network was trained normally. Could it be that the robust network was somehow "leaked"? Think about this possibility, and how it could have happen in experiment #1 and experiment #2.```

## Answer

In distillation training, one model is trained to predict the output probabilities of another model that was trained on an earlier, baseline standard to emphasize accuracy.

In the case of the first experiment, robust model is used to create a robust dataset.
We want to replace each sample x in the original dataset by a sample x_r. x and x_r must be very close in the latent space of a robust model.
We use only the representation layer of the robust model, not the output probabilities and then it is not a distillation training. Furthermore, the new sample x_r does not lie in the latent sapce, but in the input space

Experiment #2 does not use a robust model, but creates advresarial example to show that good classification can be achieved only based on non-robust features

```All in all, we will examine two conjectures:```
1. ```The paper is great, non-robust features are the real thing and different networks use the same features.```
2. ```The paper is wrong, it's only a fancy way of network distilling.```

```Let's first explore the concept of distilling networks. Can we do it in any case? Can you distill a network using only, for example, white noise? Can you do it using only the predictions? Or do you maybe need to use the logits of the network? Answer this question. MNIST might help you with that.```

In [10]:
import numpy as np
from keras.callbacks import EarlyStopping
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Layer
import tensorflow as tf

In [11]:
class DivideLayer(Layer):
    def __init__(self, temperature=1, **kwargs):
        self.temperature = temperature
        super(DivideLayer, self).__init__(**kwargs)

    def call(self, inputs, training=None):
        if training:
            return  inputs / self.temperature
        return inputs

In [12]:
def train_model(x_train, y_train, input_shape, num_classes, temperature=1):
    model = keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(0.5),
            DivideLayer(temperature),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )
    
    es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=3)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
#     model.summary()
    model.fit(x_train, y_train, batch_size=256, epochs=15, validation_split=0.1, callbacks=[es])
    
    return model

In [13]:
def TestAttack(model, adv_images, orig_images, true_labels, target_labels=None, targeted=False):
    adv_images = adv_images.numpy()
    score = model.evaluate(adv_images, true_labels, verbose=0)
    
    print('Test loss: {:.2f}'.format(score[0]))
    print('Successfully moved out of source class: {:.2f}'.format( 1 - score[1]))
    
    if targeted:
        score = model.evaluate(adv_images, target, verbose=0)
        print('Test loss: {:.2f}'.format(score[0]))
        print('Successfully perturbed to target class: {:.2f}'.format(score[1]))
    
    dist = np.mean(np.sqrt(np.mean(np.square(adv_images - orig_images), axis=(1,2,3))))
    print('Mean perturbation distance: {:.2f}'.format(dist))

In [14]:
def FastGradientSignMethod(model, input_image, input_label, eps=0.3):
    loss_object = tf.keras.losses.CategoricalCrossentropy()
    
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = model(input_image)
        loss = loss_object(input_label, prediction)

    gradient = tape.gradient(loss, input_image)
    signed_grad = tf.sign(gradient)
    adv_x = input_image + eps*signed_grad
    
    return adv_x

In [15]:
def attack_model(test_images, test_labels, model, eps=0.3):
  test_images = tf.convert_to_tensor(test_images)

  adv_images = FastGradientSignMethod(model, test_images, test_labels, eps=eps)
  TestAttack(model, adv_images, test_images.numpy(), test_labels, targeted=False)

In [24]:
def train_distillation(temperature=1, use_proba=True, white_noise_input=False):
    num_classes = 10
    input_shape = (28, 28, 1)
    
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    print("Training teacher model")
    teacher = train_model(x_train, y_train, input_shape, num_classes, temperature=temperature)
    
    _, acc = teacher.evaluate(x_test, y_test, batch_size=512)
    print('Teacher test accuracy:', acc)

    if use_proba is True:
      train_probas = teacher.predict(x_train)
    else:
      train_probas = teacher.predict(x_train)
      print(train_probas[0, :])
      train_probas = np.argmax(train_probas, axis=1)
      train_probas = tf.one_hot(train_probas, depth=num_classes)
      print(train_probas[0, :])

    if white_noise_input is True:
      x_train = np.random.normal(size=x_train.shape)

    print("Training student model")
    student = train_model(x_train, train_probas, input_shape, num_classes, temperature=temperature)
    
    _, acc = student.evaluate(x_test, y_test, batch_size=512)
    print('Student test accuracy:', acc)

    print('Attacking teacher model')
    attack_model(x_test, y_test, teacher, eps=0.3)
    print('Attacking student model')
    attack_model(x_test, y_test, student, eps=0.3)

    return teacher, student

In [20]:
teacher, student = train_distillation(temperature=10, use_proba=True)

Training teacher model
Epoch 1/15
211/211 [==============================] - 4s 14ms/step - loss: 1.3754 - accuracy: 0.6046 - val_loss: 0.7729 - val_accuracy: 0.9508
Epoch 2/15
211/211 [==============================] - 3s 12ms/step - loss: 0.2553 - accuracy: 0.9234 - val_loss: 0.5619 - val_accuracy: 0.9682
Epoch 3/15
211/211 [==============================] - 3s 13ms/step - loss: 0.1807 - accuracy: 0.9463 - val_loss: 0.4765 - val_accuracy: 0.9753
Epoch 4/15
211/211 [==============================] - 3s 12ms/step - loss: 0.1482 - accuracy: 0.9556 - val_loss: 0.4378 - val_accuracy: 0.9778
Epoch 5/15
211/211 [==============================] - 3s 12ms/step - loss: 0.1249 - accuracy: 0.9623 - val_loss: 0.4146 - val_accuracy: 0.9787
Epoch 6/15
211/211 [==============================] - 3s 12ms/step - loss: 0.1166 - accuracy: 0.9633 - val_loss: 0.3762 - val_accuracy: 0.9808
Epoch 7/15
211/211 [==============================] - 3s 12ms/step - loss: 0.1009 - accuracy: 0.9705 - val_loss: 0.3380

In [20]:
teacher, student = train_distillation(temperature=10, use_proba=False)

Training teacher model
Epoch 1/15
211/211 [==============================] - 3s 14ms/step - loss: 1.4108 - accuracy: 0.5956 - val_loss: 0.7853 - val_accuracy: 0.9508
Epoch 2/15
211/211 [==============================] - 3s 13ms/step - loss: 0.2501 - accuracy: 0.9278 - val_loss: 0.5431 - val_accuracy: 0.9707
Epoch 3/15
211/211 [==============================] - 3s 13ms/step - loss: 0.1689 - accuracy: 0.9509 - val_loss: 0.4625 - val_accuracy: 0.9770
Epoch 4/15
211/211 [==============================] - 3s 13ms/step - loss: 0.1375 - accuracy: 0.9594 - val_loss: 0.4091 - val_accuracy: 0.9792
Epoch 5/15
211/211 [==============================] - 3s 13ms/step - loss: 0.1164 - accuracy: 0.9648 - val_loss: 0.3756 - val_accuracy: 0.9822
Epoch 6/15
211/211 [==============================] - 3s 13ms/step - loss: 0.1067 - accuracy: 0.9686 - val_loss: 0.3469 - val_accuracy: 0.9830
Epoch 7/15
211/211 [==============================] - 3s 13ms/step - loss: 0.0944 - accuracy: 0.9714 - val_loss: 0.2977

In [25]:
teacher, student = train_distillation(temperature=10, use_proba=True, white_noise_input=True)

Training teacher model
Epoch 1/15
211/211 [==============================] - 4s 14ms/step - loss: 1.4245 - accuracy: 0.5812 - val_loss: 0.8294 - val_accuracy: 0.9455
Epoch 2/15
211/211 [==============================] - 3s 13ms/step - loss: 0.2729 - accuracy: 0.9210 - val_loss: 0.6061 - val_accuracy: 0.9662
Epoch 3/15
211/211 [==============================] - 3s 13ms/step - loss: 0.1865 - accuracy: 0.9436 - val_loss: 0.5152 - val_accuracy: 0.9720
Epoch 4/15
211/211 [==============================] - 3s 13ms/step - loss: 0.1464 - accuracy: 0.9571 - val_loss: 0.4520 - val_accuracy: 0.9765
Epoch 5/15
211/211 [==============================] - 3s 13ms/step - loss: 0.1251 - accuracy: 0.9621 - val_loss: 0.4054 - val_accuracy: 0.9808
Epoch 6/15
211/211 [==============================] - 3s 13ms/step - loss: 0.1143 - accuracy: 0.9661 - val_loss: 0.3943 - val_accuracy: 0.9805
Epoch 7/15
211/211 [==============================] - 3s 13ms/step - loss: 0.0993 - accuracy: 0.9699 - val_loss: 0.3717

```Distilling using only white noise is a though question. Why is it? Think about the concepts of distribution and out-of-distribution in when answering this.```

```Assuming conjecture #2 and regarding experiment #1, the distillation isn't happening on white noise, but not on real images either. Think of an experiment that will help you decide if the phenomenon the authors encountered is indeed just network distillation. Think how to measure your success. Open the hint only if you can't think of a way do to it.```

```You can, for example, take a dataset such as CIFAR10, train a network on 5 classes and try to distill it using the other 5 classes. That way you are using real images, but from different distributions.```

```Let's focus now on conjecture #1. Conduct an experiment that will demonstrate that two different networks indeed use the same features. Do it gradually: Start with two copies of the same architecture and on the same data, and move to different architectures and different subsets of the data (but from the same distribution). Think how to measure your success.```